In [1]:
import os
from langgraph.graph import StateGraph, START, END, MessagesState
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv
load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY")
BASE_URL = os.getenv("BASE_URL")

class ChatState(MessagesState):
    user_question: str
    llm_response: str

def llm_node(state):
    prompt = ChatPromptTemplate.from_messages([
        ("human", "{question}"),
    ])
    model = ChatOpenAI(model="Qwen/Qwen2.5-7B-Instruct", api_key=API_KEY, base_url=BASE_URL, temperature=0)
    chain = prompt | model
    response = chain.invoke({"question": state['user_question']}).content
    return {"llm_response": response}

# 构建图
builder = StateGraph(ChatState)
builder.add_node("llm_node", llm_node)
builder.add_edge(START, "llm_node")
builder.add_edge("llm_node", END)
graph = builder.compile()

result = graph.invoke({"user_question": "agent的发展趋势是什么？"})
print(result)

{'messages': [], 'user_question': 'agent的发展趋势是什么？', 'llm_response': '随着技术的不断进步和应用场景的不断拓展，智能代理（Agent）的发展趋势主要体现在以下几个方面：\n\n1. **智能化水平提升**：通过深度学习、强化学习等技术，智能代理能够更好地理解和适应复杂多变的环境，提高决策的准确性和效率。\n\n2. **多模态交互能力增强**：智能代理不仅能够处理文本信息，还能理解和生成语音、图像等多种形式的信息，实现更加自然和丰富的交互方式。\n\n3. **自主学习与适应能力**：智能代理能够通过自我学习不断优化自身的行为策略，适应不同的任务需求和环境变化，提高其灵活性和适应性。\n\n4. **安全性与隐私保护**：随着智能代理在更多领域的应用，如何确保其操作的安全性和保护用户隐私成为重要议题。未来的发展将更加注重数据安全和用户隐私保护。\n\n5. **跨领域应用拓展**：智能代理技术将被广泛应用于医疗健康、教育、金融、交通等多个领域，为用户提供更加个性化、高效的服务。\n\n6. **人机协作更加紧密**：智能代理将与人类更加紧密地协作，共同完成复杂任务，提高工作效率和生活质量。\n\n7. **伦理与法律框架完善**：随着智能代理技术的发展，相关的伦理和法律问题也日益凸显。未来需要建立和完善相应的伦理准则和法律法规，确保技术的健康发展。\n\n总之，智能代理技术正处于快速发展阶段，未来将朝着更加智能化、人性化、安全可靠的方向发展。'}
